In [ ]:
import sympy as sp
from IPython.display import display
sp.init_printing()

In [ ]:
x, y, a, b = sp.symbols('x y a b')

## Generace kódu

Automatická generace kódu je funkcionalita, kterou oceníme ve chvíli, kdy cheme implementovat naše analytické výsledky v numerické simulaci. Místo abychom začali ručně přepisovat do programovacího jazyka jako je např. Fortran nebo C, může SymPy tuto nezábavnou práci udělat za nás. Navíc při tom neudělá chyby.

## Převedení výrazu na Pythonovskou funkci
Pokud chceme vytvořit ze symbolického výrazu funkci, použijeme `lambdify`:

In [ ]:
# první argument je seznam proměnných (podobně jako pro lambda funkce)
f_sympy = (x + sp.pi)**2
display(f_sympy)

f_x = sp.lambdify([x], f_sympy)
print(f_x)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

xa = np.linspace(-10, 10)
plt.plot(xa, f_x(xa))

Můžeme se i podívat na zdrojový kód vytvořené funkce, to se dá pomocí modulu `inspect` a jeho funkce `getsource`.

In [ ]:
import inspect
print(inspect.getsource(f_x))

Funkce `lambdify` umí také optimalizovat výsledný kód pro snadnější exekuci. Toto se dá nastavit pomocí parametru `cse` (Common Subexpression Elimination). Tato metoda rozbije výraz na menší části, které se opakují a tyto části se pak vyhodnotí pouze jednou.

Dále lze vygenerovanou funkci specifikovat pro použití s určitým numerickým balíčkem, např. `numpy` , `jax` ... a to pomocí parametru `modules`.

In [ ]:
slozity_vyraz = (x + sp.pi + sp.exp(x))**2 + sp.sin(x) + sp.sin(x)**2 + sp.exp(x)
display(slozity_vyraz)
f1 = sp.lambdify([x], slozity_vyraz)
print(inspect.getsource(f1))
f2 = sp.lambdify([x], slozity_vyraz, cse=True, modules='numpy')
print(inspect.getsource(f2))

## Generování zdrojového kódu v jiných jazycích

SymPy může generovat zdrojový kód v několika jazycích. Využijeme k tomu modul `codegen`.

In [ ]:
import sympy.utilities.codegen as codegen

In [ ]:
# řekněme že chceme někde použít tento výsledek
f = sp.sin( x * y**2) * sp.exp(y)
f

In [ ]:
# exportujeme do jazyka Fortran
f_source = codegen.codegen(("f_fortran", f), "F95", "f_fortran")
print(f_source[0][1])

In [ ]:
# exportujeme do jazyka C
f_source = codegen.codegen(("f_C", f), "C", "f_C")
print(f_source[0][1])

In [ ]:
# exportujeme do jazyka Octave/Matlab
f_source = codegen.codegen(("f_octave", f), "Octave", "f_octave")
print(f_source[0][1])

## Ukázka na komplexnějším příkladu

V příkladu se jedná o model zobrazení pomocí sférické čočky. Tedy jak pozorovatel v prostorových souřadnicích $x,y,z$ a natočením $\alpha, \beta, \gamma$ vidí bod na pozici $S_x, S_y, S_z$.

In [ ]:
import sympy as sp
import inspect

# focal length
ff = 0.008

# length of the robot arm + length of the end effector
L = 309.5 * 1e-3

# Define the vectors v and n
v = sp.Matrix([0, 0, -1])
n = sp.Matrix([0, 1, 0])


##########################################
# Camera projection to the plane         #
##########################################

# Define the symbols
alpha, beta, gama, X, Y, Z, S_x, S_y, S_z = sp.symbols(
    'alpha beta gama X Y Z S_x S_y S_z', real=True)

# Sip and TCP
Sip = sp.Matrix([S_x, S_y, S_z])
TCP = sp.Matrix([X, Y, Z])

# Rotation matrices Rx, Ry, Rz
Rx = sp.Matrix([[1, 0, 0],
                [0, sp.cos(alpha), -sp.sin(alpha)],
                [0, sp.sin(alpha), sp.cos(alpha)]])
Ry = sp.Matrix([[sp.cos(beta), 0, sp.sin(beta)],
                [0, 1, 0],
                [-sp.sin(beta), 0, sp.cos(beta)]])
Rz = sp.Matrix([[sp.cos(gama), -sp.sin(gama), 0],
                [sp.sin(gama), sp.cos(gama), 0],
                [0, 0, 1]])
R = sp.simplify(Rx * Ry * Rz)


# Vector v and calculations for x and o
x = sp.Matrix([X, Y, Z]) - L * R * v
o = R * v

# Calculate 'a' using dot products
a = sp.sqrt(((-R.row(0).dot(v) * Z) / (R.row(2).dot(v)))**2 +
            (-(R.row(1).dot(v) * Z) / (R.row(2).dot(v)))**2 + Z * Z)

# Calculate Q
Q = a * o + TCP

# Project Sip onto the line defined by o and Q
Sip_proj = Sip + (o.dot(Q) - o.dot(Sip)) * (x - Sip) / (o.dot(x) - o.dot(Sip))

# s_2 vector and s_1 as the cross product of o and s_2
s_2 = R * n
s_1 = o.cross(s_2)

# Calculate k_1 and k_2
k_1 = (Sip_proj - Q).dot(s_1) / s_1.norm()**2
k_2 = (Sip_proj - Q).dot(s_2) / s_2.norm()**2


# Calculate image plane coordinates f_1 and f_2
f_1 = k_1 * ff / (a + L - ff)
f_2 = k_2 * ff / (a + L - ff)


vysledny_bod = sp.Matrix([f_1, f_2])
vysledny_bod

In [ ]:
# převedeme na Python funkci
kamera_py = sp.lambdify([X, Y, Z, alpha, beta, gama, S_x, S_y, S_z], vysledny_bod, modules='numpy', cse=True)
print(inspect.getsource(kamera_py))

## Další možnosti SymPy

Ukázali jsme si základy práce se symbolickými výpočty pomocí SymPy. Není v našich silách ukázat, co všechno SymPy umí -- je toho opravdu hodně. 

Přehled můžeme získat např. pomocí následujících odkazů:

* [SymPy Tutorial](http://docs.sympy.org/latest/tutorial/index.html) - základní tutoriál
* [SymPy Documentation](http://docs.sympy.org/latest/index.html) - kompletní dokumentace
* [SymPy Live](http://live.sympy.org/) - online SymPy notebook
* [SymPy GitHub](https://github.com/sympy/sympy/wiki) - SymPy na GitHubu

### Seznam témat které SymPy pokrývá

* SymPy Core
* Combinatorics Module
* Number Theory
* Concrete Mathematics
* Numerical evaluation
* Functions Module
* Geometry Module
* Geometric Algebra Module
* Geometric Algebra Module for SymPy
* Extended LaTeXModule for SymPy
* Symbolic Integrals
* Numeric Integrals
* Logic Module
* Matrices
* Mpmath
* Polynomials Manipulation Module
* Printing System
* Plotting Module
* Pyglet Plotting Module
* Assumptions module
* Term rewriting
* Series Expansions
* Sets
* Simplify
* Details on the Hypergeometric Function Expansion Module
* Statistics
* Stats
* ODE
* PDE
* Solvers
* Tensor Module
* Utilities
* Parsing input
* Physics Module
* Category Theory Module
* Differential Geometry Module
* Contributions to docs